## Collecting paints for van Gogh and various artists
This notebook is designed for collecting artworks by various artist and producing the meta info

In [1]:
import shutil
import os
import glob
import pandas as pd
import numpy
from tqdm import tqdm
import hashlib
#import wget
#from  download import *

In [2]:
!ls -alrt

total 40
drwxr-xr-x  3 czkaiweb  staff    96 May 14 16:44 .ipynb_checkpoints
drwxr-xr-x  8 czkaiweb  staff   256 May 15 15:55 ..
-rw-r--r--  1 czkaiweb  staff  2064 May 16 14:54 download.py
drwxr-xr-x  3 czkaiweb  staff    96 May 16 15:30 __pycache__
drwxr-xr-x  6 czkaiweb  staff   192 May 17 21:34 subWikiArts
-rw-r--r--@ 1 czkaiweb  staff  6148 May 18 19:26 .DS_Store
-rw-r--r--  1 czkaiweb  staff  6499 May 18 21:01 DataCollection.ipynb
drwxr-xr-x  8 czkaiweb  staff   256 May 18 21:01 .


In [3]:
# Create Meta record
fileList = []
img_path = "./imgs"

if not os.path.isdir(img_path):
    os.mkdir(img_path)


In [4]:
# Download van Gogh dataset
!kaggle datasets download -d ipythonx/van-gogh-paintings

# List all von Goph plots
tmp_path = "./tmp"
vangoghZip = 'van-gogh-paintings.zip'
try:
    shutil.unpack_archive(vangoghZip,tmp_path)
except Exception as err:
    print(err)

allVanGogh = glob.glob(tmp_path+'/*/*.jpg')

# Append metadata
for index in tqdm(range(len(allVanGogh))):
    fileName = allVanGogh[index]
    file = fileName.split("/")[-1]
    hashName = hashlib.md5(file.encode()).hexdigest()
    shutil.move(fileName, img_path + "/" + hashName + ".jpg", copy_function = shutil.copy2)
    artist = "vanGogh"
    fileList.append([hashName,artist])
    
# Clean tmp data
try:
    shutil.rmtree(tmp_path)
    os.remove(vangoghZip)
except Exception as err:
    print(err)

100%|███████████████████████████████████████▊| 483M/485M [00:15<00:00, 45.1MB/s]
100%|████████████████████████████████████████| 485M/485M [00:15<00:00, 33.5MB/s]


100%|██████████| 2024/2024 [00:00<00:00, 4433.10it/s]


In [5]:
# Download Monet dataset: https://www.kaggle.com/datasets/srrrrr/monet2photo
!kaggle datasets download -d srrrrr/monet2photo

# List all Monet plots
tmp_path = "./tmp"
monetZip = 'monet2photo.zip'
try:
    shutil.unpack_archive(monetZip,tmp_path)
except Exception as err:
    print(err)

allMonet = glob.glob(tmp_path+'/*/trainA/*.jpg')

# Append metadata
for index in tqdm(range(len(allMonet))):
    fileName = allMonet[index]
    file = fileName.split("/")[-1]
    hashName = hashlib.md5(file.encode()).hexdigest()
    shutil.move(fileName, img_path + "/" + hashName + ".jpg", copy_function = shutil.copy2)
    artist = "Monet"
    fileList.append([hashName,artist])
    
# Clean tmp data
try:
    shutil.rmtree(tmp_path)
    os.remove(monetZip)
except Exception as err:
    print(err)

100%|████████████████████████████████████████| 263M/263M [00:07<00:00, 21.6MB/s]
100%|████████████████████████████████████████| 263M/263M [00:07<00:00, 37.5MB/s]


100%|██████████| 1072/1072 [00:00<00:00, 7713.07it/s]


In [9]:
# Download WikiArts dataset: https://www.kaggle.com/datasets/antoinegruson/-wikiart-all-images-120k-link
!kaggle datasets download -d czkaiweb/subwikiarts

# List all wikiarts plots
tmp_path = "./tmp"
wikiartsZip = 'subwikiarts.zip'
try:
    shutil.unpack_archive(wikiartsZip,tmp_path)
except Exception as err:
    print(err)

WikiArtsMeta = tmp_path+"/WikiArts.csv"
WikiArtsDF = pd.read_csv(WikiArtsMeta)
WikiArtsList = WikiArtsDF[["hash","Artist"]].values

def findGroup(head):
    if head <= "33":
        return "/GroupA/"
    elif head <= "69":
        return "/GroupB/"
    elif head <= "9d":
        return "/GroupC/"
    elif head <= "cc":
        return "/GroupD/"
    else:
        return "/GroupE/"
    
# Set to true for group splitting
preClean = False
if preClean == True:
    for char in ["A","B","C","D","E"]:
        groupDir = img_path+"/Group{}".format(char)
        if not os.path.isdir(groupDir):
            os.mkdir(groupDir)

for record in WikiArtsList:
    groupDir = "/./"
    if preClean:
        groupDir = findGroup(record[0][:2])
    fileName = tmp_path+"/imgs/"+record[0]+".jpg"
    shutil.move(fileName, img_path+ groupDir + "/" , copy_function = shutil.copy2)
    
# Clean tmp data
try:
    shutil.rmtree(tmp_path)
    os.remove(wikiartsZip)
except Exception as err:
    print(err)


100%|███████████████████████████████████████▉| 993M/995M [00:49<00:00, 38.1MB/s]
100%|████████████████████████████████████████| 995M/995M [00:49<00:00, 21.2MB/s]


In [10]:
# Save to meta file
metaDF = pd.DataFrame(fileList,columns = ["hash","Artist"])
metaDF = pd.concat([metaDF,WikiArtsDF[["hash","Artist"]]])
metaDF.to_csv("meta.csv")